In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [6]:
!ls "/content/gdrive/My Drive/dataset"

with_mask  without_mask


In [11]:
import cv2, os

data_path="/content/gdrive/My Drive/dataset"
categories=os.listdir(data_path)[::-1]
labels=[i for i in range(len(categories))]
labels_dict=dict(zip(categories,labels))

print(categories)
print(labels)
print(labels_dict)

['with_mask', 'without_mask']
[0, 1]
{'with_mask': 0, 'without_mask': 1}


In [13]:
img_size=100
data=[]
target=[]

for category in categories:
  folder_path=os.path.join(data_path, category)
  img_names=os.listdir(folder_path)

  for img_name in img_names:
    img_path=os.path.join(folder_path,img_name)
    img=cv2.imread(img_path)

    try:
      #converting the image into grayscale
      gray_img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      #resizing the grayscale image to 100x100 (common fixed size for all images)
      resized_img=cv2.resize(gray_img,(img_size,img_size))
      data.append(resized_img)
      target.append(labels_dict[category])

    except Exception as e:
      print('Exception:', e)


In [14]:
print(len(data))
print(len(target))

1376
1376


In [15]:
import numpy as np

data=np.array(data)/255.0
print(data.shape)
data=np.reshape(data, (data.shape[0],img_size,img_size,1))
print(data.shape)
target = np.array(target)
print(target.shape)

(1376, 100, 100)
(1376, 100, 100, 1)
(1376,)


In [20]:
from keras.utils import np_utils

new_target=np_utils.to_categorical(target)
print(new_target.shape)
print(new_target[0])

(1376, 2)
[1. 0.]


In [33]:
np.save('data',data)
np.save('target',new_target)

In [37]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint

model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(50,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [38]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_target, test_target=train_test_split(data,new_target,test_size=0.2)
print(train_data.shape)
print(test_data.shape)
print(train_target.shape)
print(test_target.shape)

(1100, 100, 100, 1)
(276, 100, 100, 1)
(1100, 2)
(276, 2)


In [39]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model', monitor='val_loss', verbose=0, save_best_only=True, mode='auto')
model.fit(train_data, train_target, epochs=20, callbacks=[checkpoint], validation_split=0.2)

Epoch 1/20
28/28 [==============================] - 67s 2s/step - loss: 0.7414 - accuracy: 0.4977 - val_loss: 0.6621 - val_accuracy: 0.6909
Epoch 2/20
28/28 [==============================] - 67s 2s/step - loss: 0.6339 - accuracy: 0.6318 - val_loss: 0.5447 - val_accuracy: 0.6955
Epoch 3/20
28/28 [==============================] - 70s 3s/step - loss: 0.4872 - accuracy: 0.7795 - val_loss: 0.3797 - val_accuracy: 0.8455
Epoch 4/20
28/28 [==============================] - 66s 2s/step - loss: 0.3406 - accuracy: 0.8614 - val_loss: 0.2998 - val_accuracy: 0.8909
Epoch 5/20
28/28 [==============================] - 67s 2s/step - loss: 0.2502 - accuracy: 0.9170 - val_loss: 0.2247 - val_accuracy: 0.9136
Epoch 6/20
28/28 [==============================] - 67s 2s/step - loss: 0.1977 - accuracy: 0.9341 - val_loss: 0.2065 - val_accuracy: 0.9227
Epoch 7/20
28/28 [==============================] - 66s 2s/step - loss: 0.1574 - accuracy: 0.9443 - val_loss: 0.2238 - val_accuracy: 0.9364
Epoch 8/20
28/28 [==

In [40]:
print(model.evaluate(test_data,test_target))

9/9 [==============================] - 5s 592ms/step - loss: 0.1478 - accuracy: 0.9493
[0.14781707525253296, 0.9492753744125366]


In [41]:
model.save_weights('model.h5')

In [47]:
!zip -r /content/file.zip /content/model-015.model

  adding: content/model-015.model/ (stored 0%)
  adding: content/model-015.model/variables/ (stored 0%)
  adding: content/model-015.model/variables/variables.index (deflated 63%)
  adding: content/model-015.model/variables/variables.data-00000-of-00001 (deflated 17%)
  adding: content/model-015.model/saved_model.pb (deflated 89%)
  adding: content/model-015.model/assets/ (stored 0%)


In [50]:
from keras.models import load_model
model = load_model('model-015.model')

face_clf=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

labels_dict={0:"MASK",1:"NO MASK"}
color_dict={0:(0,255,0),1:(0,0,255)}

In [51]:
source=cv2.VideoCapture(0)
while(True):
    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clf.detectMultiScale(gray,1.3,5)  

    for (x,y,w,h) in faces:
    
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=model.predict(reshaped)

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()
  

error: ignored